In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from collections import defaultdict
import codecs, json
import unicodedata
# pip install Unidecode  <OR> conda install Unidecode
import unidecode
import collections
import datetime

In [11]:
## Variables and what they mean
#   duplicate_dict = all duplicates
#   dict_removed_single_entries = all duplicates but removed single entries
#   duplicate_ids_kept = array of all ids from dict_removed_single_entries
#   dict_duplicate_compare_team_members = map values from teams to einstaklingsid
#   dict_name_entries = map values from member down one step (name->birthday->values) now (name+birthday->values)
#   dict_einstaklingar_teammember_info = map teammember values to correct key in einstaklingsid
#   not_the_same_person = when two players are playing in different teams at the same time then they clearly are not the same person

# Import CSV

In [3]:
#importing all csv files
domarar = pd.read_csv('csv/blak-domarar.csv', sep=';', header=0)
einstaklingar = pd.read_csv('csv/blak-einstaklingar.csv', sep=';', header=0)
forsvarsmenn = pd.read_csv('csv/blak-forsvarsmenn.csv', sep=';', header=0)
lid = pd.read_csv('csv/blak-lid.csv', sep=';', header=0)
lidimoti = pd.read_csv('csv/blak-lidimoti.csv', sep=';', header=0)
lidsmenn = pd.read_csv('csv/blak-lidsmenn.csv', sep=';', header=0)
lidsstjorar = pd.read_csv('csv/blak-lidsstjorar.csv', sep=';', header=0)
thjalfarar = pd.read_csv('csv/blak-thjalfarar.csv', sep=';', header=0)
mot = pd.read_csv('csv/blak-mot.csv', sep=';', header=0)

# drop all SyndarLids with an ID (SyndarlidID)
# (the reason for not dropping using SyndarLid is because I don't trust that column to be inserted correctly with [0,1])
lid = lid[lid['SyndarlidID'].isna()]
# then dropping those two columns because we don't want virtual teams
lid = lid.drop(columns=['SyndarLid', 'SyndarlidID'])

# All duplicated birthdays
duplicated_einstaklingar = einstaklingar[einstaklingar.duplicated(subset=['Nafn', 'Fdagur', 'Kyn'], keep=False)]
duplicated_fdagur_kyn_einstaklingar = einstaklingar[einstaklingar.duplicated(subset=['Fdagur', 'Kyn'], keep=False)]


# Filter duplicates by name and birthday

In [4]:
# Add all entries that have duplicated birthdays, then filter that to first_name->birthday-><people entries>
duplicate_dict = defaultdict(dict)
for index, row in duplicated_fdagur_kyn_einstaklingar.iterrows():
    full_name = row['Nafn']
    #only get the first part of full name 
    first_name = full_name.split()[0]
    # make first name lowercase
    first_name_lowercase = first_name.lower()
    # encode icelandic letters to english
    first_name_to_english = unidecode.unidecode(first_name_lowercase)
    # split birthday into year month and day and ignore second part (sec, min, hour)
    Fdagur_date = row['Fdagur'].split()[0]
    
    if first_name_to_english in duplicate_dict.keys():
        if Fdagur_date in duplicate_dict[first_name_to_english].keys():
            #if first name and Fdagur (birthday) exist in dict then append to that key (birthday)
            duplicate_dict[first_name_to_english][Fdagur_date].append(row.values)
        else:
            #if first name exists but Fdagur (birthday) does not exist in dict
            duplicate_dict[first_name_to_english][Fdagur_date] = [row.values]
    else:
        #if Fdagur (birthday) does not exist in dict
        duplicate_dict[first_name_to_english][Fdagur_date] = [row.values]
        

# Remove all single birthday entries (since that is not a duplicate)

In [5]:
# Remove all single birthday entries that are not duplicates
dict_removed_single_entries = defaultdict(dict)

for key, values in duplicate_dict.items():
    # key = nafn ('ludvik')
    for birthday, arrays in dict(values).items():
        # only get duplicates that there exists 2 or more entries for a birthday
        if(len(arrays) > 1):
            # used for when joining teams table
            if key in dict_removed_single_entries.keys():
                if birthday in dict_removed_single_entries[key].keys():
                    dict_removed_single_entries[key][birthday].append(arrays)
                else:
                    #if first name exists but Fdagur (birthday) does not exist in dict
                    dict_removed_single_entries[key][birthday] = arrays
            else:
                dict_removed_single_entries[key][birthday] = arrays

#dict_removed_single_entries

# Get all ids that exists in duplication

In [6]:
# get all ids in dict_removed_single_entries
duplicate_ids_kept = []
for key, values in dict_removed_single_entries.items():
    # key = nafn ('ludvik')
    for birthday, arrays in dict(values).items():
        for item in arrays:
            duplicate_ids_kept.append(item[0])

# Map teams table values to einstaklingsID

In [7]:
# Checking if two names are the same person
dict_duplicate_compare_team_members = defaultdict(dict)
for index, row in lidsmenn.iterrows():
    ids = row["EinstID"]
    if ids in duplicate_ids_kept:
        # now we only view ids that exist for duplicated people
        #print(ids)
        if ids in dict_duplicate_compare_team_members.keys():
            dict_duplicate_compare_team_members[ids].append(row.values)
        else:
            dict_duplicate_compare_team_members[ids] = [row.values]


## Map one step down (name->birthday -> value) now (name+birthday -> value)

In [8]:
dict_name_entries = {}
for key, value in dict_removed_single_entries.items():
    #get key and arrays for each person
    for birthday, arrays in dict(value).items():
        #get each array for person
        #print("KEY: " + key + " BIRTHDAY: " + birthday)
        new_key = key +"-"+ birthday
        for item in arrays:
            if new_key in dict_name_entries.keys():
                dict_name_entries[new_key].append(item[0])
            else:
                dict_name_entries[new_key] = [item[0]]
#dict_name_entries

## EinstaklingsID+birthday connected to all his data from teams table 

In [43]:
dict_einstaklingar_teammember_info = {}
for key, value in dict_name_entries.items():
    #print("<key>" + str(key) + " <value> " + str(value))
    for item in value:
        #print(item)
        if item in dict_duplicate_compare_team_members.keys():
            #print("<key>" + str(key) + " <item> " + str(item))
            for compare_arrays in dict_duplicate_compare_team_members[item]:
                mot_id = compare_arrays[0]
                lid_id = compare_arrays[1]
                player_id = compare_arrays[2]
                date = compare_arrays[3]
                date_played = compare_arrays[3].split()[0]
                
                temp = (str(date) + " " + str(mot_id) + " " + str(lid_id) + " " + str(player_id))   
                if key in dict_einstaklingar_teammember_info.keys():
                    dict_einstaklingar_teammember_info[key].append(temp)
                else:
                    dict_einstaklingar_teammember_info[key] = [temp]
        #print("xxxxxxxxxxxx")
#dict_einstaklingar_teammember_info

## Find if a potential duplicated person played two games at the same time in different teams (then he is not a duplication)

In [44]:
not_the_same_person = {}

def find_duplicates(key, nums):
    num_set = set()
    duplicates = set()
    no_duplicate = -1
    sorted_nums = sorted(nums)
    
    for i in range(len(sorted_nums)):
        for j in range(i+1, len(sorted_nums)):
            
            # team one split
            #(str(date) + " " + str(mot_id) + " " + str(lid_id) +  str(player_id))   
            
            sort_1 = sorted_nums[i].split()
            date_1 = sort_1[0]
            mot_id_1 = sort_1[2]
            team_id_1 = sort_1[3]
            einstaklings_id_1 = sort_1[4]
            date_time_str_1 = sort_1[0]+" "+sort_1[1]
            date_time_obj_1 = datetime.datetime.strptime(date_time_str_1, '%Y-%m-%d %H:%M:%S.%f')
            
            # team two split
            sort_2 = sorted_nums[j].split()
            date_2 = sort_2[0]
            mot_id_1 = sort_2[2]
            team_id_2 = sort_2[3]
            einstaklings_id_2 = sort_2[4]    
            date_time_str_2 = sort_2[0]+" "+sort_2[1]
            date_time_obj_2 = datetime.datetime.strptime(date_time_str_2, '%Y-%m-%d %H:%M:%S.%f')
            
            # time difference between these two entries
            time_diff = (date_time_obj_2 - date_time_obj_1).total_seconds()/60
            
            match_length = 60
            if((date_1 == date_2) and (team_id_1 != team_id_2) and (time_diff < match_length)):
                # There exist two record for erla with the same einstaklingsid but different teams (6286 and 6285)
                # played 6.5 minutes apart
                # TODO: figure out how to handle that
                if(einstaklings_id_1 != einstaklings_id_2):
                    print("=======================")
                    print("SAME ID")
                    print("Time diff: " + str(time_diff))
                    print("Key: " + key)
                    print("Row 1: " + sorted_nums[i])
                    print("Row 2: " + sorted_nums[j])
             
    
for key, value in dict_einstaklingar_teammember_info.items():
    #print(value)
    find_duplicates(key, value)

['2011-01-20 21:19:21.280 187 2840 1964', '2013-09-03 20:55:14.783 232 1022 1964', '2013-09-09 15:35:13.947 234 1022 1964', '2014-04-21 21:12:11.077 249 2840 1964', '2004-07-26 12:25:47.497 45 1023 1437', '2003-10-11 19:12:00.843 46 1023 1437', '2004-09-17 11:59:19.497 59 1023 1437', '2004-09-30 17:47:04.420 64 1023 1437', '2004-11-15 23:01:41.150 66 1015 1437', '2005-02-23 18:40:23.027 70 2085 1437', '2005-10-01 20:06:49.467 84 1023 1437', '2010-09-27 10:28:20.920 176 1022 1437', '2011-08-29 20:53:45.547 191 1022 1437', '2012-03-04 14:10:07.770 199 2840 1437', '2012-04-04 20:14:56.530 201 2690 1437', '2012-01-13 00:28:42.140 212 5601 1437', '2012-07-06 10:43:22.870 215 1022 1437', '2013-03-27 19:40:58.193 221 2840 1437', '2012-10-09 10:49:09.633 222 1022 1437', '2013-03-27 19:42:14.360 225 2840 1437', '2013-03-13 21:09:03.820 230 2840 1437', '2014-08-14 10:15:41.083 250 1830 1437']
['2010-02-25 13:52:40.360 169 4378 2949', '2010-02-25 13:52:42.780 169 4378 2950']
['2004-08-20 23:05:27

['2002-04-29 10:53:07.390 15 171 1159', '2003-03-24 16:35:29.593 34 171 1159', '2003-02-07 17:41:56.530 39 171 1159', '2003-11-16 13:10:11.670 44 171 1159', '2004-03-27 15:02:47.153 54 171 1159', '2005-04-03 23:00:12.500 68 171 1159', '2006-11-08 13:07:25.200 97 171 1159', '2006-12-27 21:12:13.577 107 171 1159', '2009-04-02 22:33:20.060 138 2972 1159', '2010-04-14 14:57:55.513 165 2972 1159', '2011-03-31 11:29:46.390 189 88 1159', '2014-04-17 10:32:48.997 249 88 1159', '2013-10-22 13:33:35.153 232 4156 4212']
['2001-04-14 16:44:57.607 2 147 796', '2002-04-11 17:23:33.310 15 147 796', '2003-04-01 14:10:46.357 34 147 796', '2004-04-14 10:20:07.077 54 147 796', '2007-01-05 10:29:01.793 108 145 796', '2008-04-07 14:20:13.200 121 147 796', '2009-04-21 20:13:37.890 138 2216 796', '2010-04-11 18:03:12.483 165 147 796', '2012-03-08 10:52:38.377 199 147 796', '2012-02-22 21:27:50.517 208 2217 796', '2005-04-12 10:37:51.670 68 147 1758', '2006-04-17 18:07:09.250 91 1567 1758', '2007-04-03 17:09:

['2008-04-29 09:12:24.403 121 2579 2470', '2009-03-25 09:25:37.670 138 3606 2470', '2010-05-08 20:25:50.780 165 3606 2470', '2011-03-16 14:38:54.187 189 4256 2470', '2012-03-28 23:29:36.327 199 4976 3758']
['2012-04-25 22:35:22.840 199 4976 3845', '2000-04-03 16:42:44.100 1 42 606', '2001-04-17 12:19:53.353 2 42 606', '2002-04-15 23:23:22.500 15 42 606', '2003-01-07 22:45:46.710 28 42 606', '2003-03-31 23:29:14.233 34 42 606', '2004-04-20 09:29:08.420 54 42 606', '2005-04-04 08:41:46.310 68 1837 606', '2006-03-30 11:41:39.153 91 1837 606', '2007-04-02 22:27:18.903 109 673 606', '2013-04-09 14:31:40.183 230 4975 606', '2014-04-10 11:49:24.120 249 4975 606']
['2007-10-10 09:39:32.280 113 3034 2337', '2013-11-07 19:29:59.110 232 5997 2337']
['2011-01-31 23:00:50.310 182 1256 3224', '2011-01-12 09:48:46.450 187 4664 3224', '2012-04-11 13:51:55.530 201 4155 3224', '2013-04-23 09:28:33.357 230 4701 3224', '2010-04-15 09:50:17.780 165 4447 3098', '2010-10-14 09:56:47.390 172 4155 3098', '2010

['2010-09-27 08:23:42.013 176 1009 3214']
['2009-04-20 14:56:57.140 138 43 1152', '2009-07-19 16:29:27.437 144 3868 1152', '2012-04-23 09:33:27.190 199 43 1152', '2014-04-10 11:49:45.993 249 4975 1152', '2002-05-06 10:08:13.593 15 43 1181', '2004-04-19 19:39:59.560 54 42 1181', '2005-04-19 23:02:12.170 68 1693 1458', '2008-03-28 17:49:57.950 121 363 1458', '2000-04-29 11:41:13.653 1 36 131', '2001-04-07 21:25:20.940 2 36 131', '2003-01-09 09:47:01.737 28 36 131', '2003-03-31 22:38:47.810 34 36 131', '2003-11-04 13:57:38.340 43 1460 131', '2004-01-05 20:07:30.450 49 36 131', '2004-11-01 16:40:00.233 67 1460 131']
['2000-04-04 11:02:28.890 1 84 253', '2001-04-09 14:08:13.740 2 84 253', '2001-12-20 10:32:03.437 8 84 253', '2002-02-28 15:43:51.403 11 84 253', '2002-04-05 12:13:21.327 15 84 253', '2002-11-08 12:01:57.980 26 84 253', '2004-03-30 22:54:39.810 54 84 253', '2003-03-13 09:08:24.890 34 84 1038']
['2010-04-16 10:01:14.467 155 4448 3101', '2010-10-22 10:08:10.280 172 4635 3101', '2

['2011-11-22 10:40:02.987 193 2865 1958', '2000-04-04 15:32:58.707 1 102 476', '2001-04-11 22:31:11.437 2 102 476', '2002-03-26 16:14:33.857 15 102 476', '2003-03-27 23:48:53.577 34 102 476', '2003-11-25 13:25:11.983 44 463 476', '2004-01-06 20:30:35.233 49 463 476', '2004-03-30 23:24:41.810 54 102 476', '2004-11-15 22:07:12.593 66 463 476', '2005-04-06 12:38:11.123 68 463 476', '2006-04-02 16:58:51.030 91 463 476', '2007-04-02 17:16:23.547 109 2979 476', '2009-03-30 22:22:10.310 138 3451 476', '2010-04-14 20:25:47.420 165 3451 476', '2012-04-22 21:34:28.083 199 4193 476', '2014-04-18 10:35:53.853 249 4193 476']
['2013-09-20 15:55:26.220 232 1444 4204', '2013-11-04 22:57:59.253 233 1444 4204', '2014-04-02 23:20:26.057 235 4168 4204', '2013-10-25 18:48:18.870 232 4168 4287']
['2006-04-11 08:08:17.530 91 2690 2075', '2007-03-05 20:13:42.780 109 2690 2075', '2008-03-15 22:59:39.123 121 2690 2075', '2009-03-14 20:32:04.530 138 2690 2075', '2010-03-30 20:25:02.903 155 2690 2075']
['2000-04-

SAME ID
Time diff: 11.125
Key: olafur-1959-05-06
Row 1: 2005-04-18 22:40:16.903 68 1689 1497
Row 2: 2005-04-18 22:51:24.403 68 38 722
SAME ID
Time diff: 11.899433333333333
Key: olafur-1959-05-06
Row 1: 2008-04-24 09:15:58.937 121 2980 722
Row 2: 2008-04-24 09:27:52.903 121 38 1497
SAME ID
Time diff: 4.895266666666667
Key: olafur-1959-05-06
Row 1: 2010-05-05 20:55:47.937 165 2980 1497
Row 2: 2010-05-05 21:00:41.653 165 1689 722
['2013-03-12 22:50:10.703 230 1954 4021', '2014-03-09 00:40:23.480 249 1954 4021', '2010-02-10 11:12:37.153 165 1021 2913']
['2007-03-13 22:32:20.280 109 2690 2175', '2008-03-15 22:59:39.123 121 2690 2175', '2009-03-14 20:32:04.530 138 2690 2175', '2010-03-30 20:25:02.903 155 2690 2175']
['2005-04-09 12:03:21.530 68 91 1824', '2006-04-03 15:06:08.390 91 91 1824', '2007-04-23 09:59:53.263 109 1693 1824', '2008-04-05 17:41:52.013 121 1130 1824', '2010-04-14 19:50:37.123 165 1044 1824', '2006-01-10 15:28:28.357 86 1045 1967', '2007-11-13 09:52:28.733 114 91 1967', '

['2010-02-25 21:54:36.077 169 4401 3013', '2010-02-25 21:54:38.937 169 4401 3014']
['2012-03-13 09:18:34.543 199 4256 3661', '2013-03-18 12:28:03.317 230 3606 3661', '2014-03-31 08:43:32.003 249 3606 4478']
['2001-04-15 15:14:47.500 2 350 837', '2003-04-21 18:53:17.590 34 1088 837', '2003-11-24 21:49:37.450 44 153 837', '2004-03-30 22:28:05.123 54 153 837', '2005-01-04 20:28:16.230 71 1087 837', '2006-04-03 11:51:07.810 91 1688 837', '2007-03-28 15:09:55.827 109 1688 837', '2008-04-11 08:50:45.233 121 1688 837', '2009-11-23 14:03:32.903 157 4167 837', '2010-01-07 17:51:36.200 161 350 837', '2011-04-04 21:57:43.543 189 153 837', '2012-04-20 11:57:36.927 199 350 837', '2013-03-21 11:49:13.647 230 350 837', '2014-04-26 20:53:38.137 249 1088 837', '2009-04-03 16:54:19.043 138 1087 1327', '2010-04-14 09:30:10.090 165 1087 1327']
['2007-07-03 21:12:53.403 111 3010 2309', '2008-08-31 19:49:42.450 127 3426 2486']
['2013-04-15 23:59:29.917 230 587 4124', '2014-03-24 22:41:36.553 249 587 4124', 

=====================================================================================
=

In [ ]:
#reverted_back_to_dict = dict(duplicate_dict)
#reverted_back_to_dict
#file_path = "json/einstaklingar_map.txt" ## your path variable
#duplicate_dict_json = json.dump(duplicate_dict, codecs.open(file_path, 'w', encoding='utf-8'), separators=(';', ':'), sort_keys=True, indent=4) ### this saves the array in .json format
#json_obj = json.dumps(duplicate_dict, indent = 4)
#dumped = json.dumps(duplicate_dict, cls=NumpyEncoder)
#dumped
#pd.DataFrame(reverted_back_to_dict).to_csv(file_path, encoding='utf-8-sig')
#duplicate_dict_json = json.dump(reverted_back_to_dict, codecs.open(file_path, 'w', encoding='utf-8-sig'))

#json = json.dumps(reverted_back_to_dict)
#f = open(file_path,"w")
#f.write(str(reverted_back_to_dict))
#f.close()

=====================================================================================
=

In [ ]:
#FINAL STEP (run after everything is done):

#duplicated people put into it's own csv to be browsed later
pd.DataFrame(duplicated_einstaklingar).to_csv("csv/new/duplicated-einstaklingar.csv", encoding='utf-8-sig')
pd.DataFrame(duplicate_dict).to_csv("json/duplicate-map.json", encoding='utf-8-sig')


#save as new csv inside csv/new
pd.DataFrame(domarar).to_csv("csv/new/blak-domarar.csv", encoding='utf-8-sig')
pd.DataFrame(einstaklingar).to_csv("csv/new/blak-einstaklingar.csv", encoding='utf-8-sig')
pd.DataFrame(forsvarsmenn).to_csv("csv/new/blak-forsvarsmenn.csv.csv", encoding='utf-8-sig')
pd.DataFrame(lid).to_csv("csv/new/blak-lid.csv", encoding='utf-8-sig')
pd.DataFrame(lidimoti).to_csv("csv/new/blak-lidimoti.csv", encoding='utf-8-sig')
pd.DataFrame(lidsmenn).to_csv("csv/new/blak-lidsmenn.csv", encoding='utf-8-sig')
pd.DataFrame(lidsstjorar).to_csv("csv/new/blak-lidsstjorar.csv", encoding='utf-8-sig')
pd.DataFrame(mot).to_csv("csv/new/blak-mot.csv", encoding='utf-8-sig')
pd.DataFrame(thjalfarar).to_csv("csv/new/blak-thjalfarar.csv", encoding='utf-8-sig')